In [1]:
!pip install librosa
!pip install mir_eval

     |████████████████████████████████| 90 kB 4.1 MB/s 
  Created wheel for mir-eval: filename=mir_eval-0.7-py3-none-any.whl size=100721 sha256=54f88327b13eeecdb1f91c1353278f97734d66cf0e466ff201daf4137d85e032
  Stored in directory: /root/.cache/pip/wheels/18/5a/46/d2527ff1fd975e1a793375e6ed763bfe4d3ea396b7cdc470eb
Successfully built mir-eval


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import librosa
import librosa.display as dsp
import mir_eval
from IPython.display import Audio
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import math
import numpy as np
from PIL import Image
import joblib
import cv2
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf 
import random
from keras.models import Sequential
from keras.models import Model
from keras import initializers
from keras import optimizers
# from keras.utils import plot_model
from keras.layers import *
import pandas as pd
from tqdm.notebook import tqdm
import pathlib
import io
from six.moves.urllib.request import urlopen
import warnings
warnings.filterwarnings("ignore")

In [4]:
def create_spectrogram_file(inp_file):
    y, sr = librosa.load(inp_file)
    melspectrogram_array = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)
    mel = librosa.power_to_db(melspectrogram_array)
    # Length and Width of Spectogram
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = float(mel.shape[1]) // float(100)
    fig_size[1] = float(mel.shape[0]) // float(100)
    plt.rcParams["figure.figsize"] = fig_size
    plt.axis('off')
    plt.axes([0., 0., 1., 1.0], frameon=False, xticks=[], yticks=[])
    librosa.display.specshow(mel, cmap='gray_r')
    op_file = "/content/drive/MyDrive/IR Project/Test_Spectogram_Images/"+inp_file.split("/")[-1].split(".")[0]+".jpg"
    plt.savefig(op_file, bbox_inches=None, pad_inches=0)
    plt.close()
    return op_file
    
def create_spectrogram(verbose=0, mode=None, continue_=6647):
    if mode == "Train":
        if os.path.exists('/content/drive/MyDrive/IR Project/Train_Spectogram_Images'):
            return
        # Get Genres and Track IDs from the tracks.csv file
        filename_metadata = "/content/drive/MyDrive/IR Project/Data/fma_metadata/tracks.csv"
        tracks = pd.read_csv(filename_metadata, header=2, low_memory=False)
        tracks_array = tracks.values
        tracks_id_array = tracks_array[: , 0]
        tracks_genre_array = tracks_array[: , 40]
        tracks_id_array = tracks_id_array.reshape(tracks_id_array.shape[0], )
        tracks_genre_array = tracks_genre_array.reshape(tracks_genre_array.shape[0], )
        folder_sample = "/content/drive/MyDrive/IR Project/Data/fma_small"
        directories = [d for d in os.listdir(folder_sample)
                       if os.path.isdir(os.path.join(folder_sample, d))]
        counter = 0
        if(verbose > 0):
            print("Converting mp3 audio files into mel Spectograms ...")
        if not os.path.exists('/content/drive/MyDrive/IR Project/Train_Spectogram_Images'):
            os.makedirs('/content/drive/MyDrive/IR Project/Train_Spectogram_Images')
        labels = [os.path.join(folder_sample, d) for d in directories]
        total = 0
        for l in labels:
            total += len([f for f in os.listdir(l) if f.endswith(".mp3")])
        with tqdm(total = total) as pbar:
            for d in directories:
                label_directory = os.path.join(folder_sample, d)
                file_names = [os.path.join(label_directory, f)
                            for f in os.listdir(label_directory)
                            if f.endswith(".mp3")]

                # Convert .mp3 files into mel-Spectograms
                for f in file_names:
                    counter += 1
                    pbar.update(1)
                    if counter <= continue_:
                        continue
                    track_id = int(re.search('/content/drive/MyDrive/IR Project/Data/fma_small/.*/(.+?).mp3', f).group(1))
                    try:
                        track_index = list(tracks_id_array).index(track_id)
                    except ValueError:
                        print(f"Error: Track ID {track_id} not found in tracks.csv file.")
                        continue
                    if(str(tracks_genre_array[track_index]) != '0'):
                        try:
                            y, sr = librosa.load(f)
                        except Exception:
                            print(f"Couldn't convert {track_id}")
                        melspectrogram_array = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)
                        mel = librosa.power_to_db(melspectrogram_array)
                        # Length and Width of Spectogram
                        fig_size = plt.rcParams["figure.figsize"]
                        fig_size[0] = float(mel.shape[1]) // float(100)
                        fig_size[1] = float(mel.shape[0]) // float(100)
                        plt.rcParams["figure.figsize"] = fig_size
                        plt.axis('off')
                        plt.axes([0., 0., 1., 1.0], frameon=False, xticks=[], yticks=[])
                        librosa.display.specshow(mel, cmap='gray_r')
                        plt.savefig("/content/drive/MyDrive/IR Project/Train_Spectogram_Images/"+str(counter)+"_"+str(tracks_genre_array[track_index])+".jpg", bbox_inches=None, pad_inches=0)
                        plt.close()
                    
        return

    elif mode == "Test":
        if os.path.exists('/content/drive/MyDrive/IR Project/Test_Spectogram_Images'):
            return

        folder_sample = "/content/drive/MyDrive/IR Project/Data/DLMusicTest_30"
        counter = 0
        if(verbose > 0):
            print("Converting mp3 audio files into mel Spectograms ...")
        if not os.path.exists('/content/drive/MyDrive/IR Project/Test_Sepctogram_Images'):
            os.makedirs('/content/drive/MyDrive/IR Project/Test_Spectogram_Images')
        file_names = [os.path.join(folder_sample, f) for f in os.listdir(folder_sample)
                       if f.endswith(".mp3")]
        # Convert .mp3 files into mel-Spectograms
        for f in file_names:
            test_id = re.search('/content/drive/MyDrive/IR Project/Data/DLMusicTest_30/(.+?).mp3', f).group(1)
            y, sr = librosa.load(f)
            melspectrogram_array = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)
            mel = librosa.power_to_db(melspectrogram_array)
            # Length and Width of Spectogram
            fig_size = plt.rcParams["figure.figsize"]
            fig_size[0] = float(mel.shape[1]) // float(100)
            fig_size[1] = float(mel.shape[0]) // float(100)
            plt.rcParams["figure.figsize"] = fig_size
            plt.axis('off')
            plt.axes([0., 0., 1., 1.0], frameon=False, xticks=[], yticks=[])
            librosa.display.specshow(mel, cmap='gray_r')
            plt.savefig("/content/drive/MyDrive/IR Project/Test_Spectogram_Images/"+test_id+".jpg", cmap='gray_r', bbox_inches=None, pad_inches=0)
            plt.close()
        return

In [5]:
def slice_spect_file(inp_file):
    img = Image.open(inp_file)
    subsample_size = 128
    width, height = img.size
    number_of_samples = width // subsample_size
    op_files = []
    counter = 0
    for i in range(number_of_samples):
        start = i*subsample_size
        img_temporary = img.crop((start, 0., start + subsample_size, subsample_size))
        op_file = "/content/drive/MyDrive/IR Project/Test_Sliced_Images/"+str(counter)+"_"+inp_file.split("/")[-1]
        img_temporary.save(op_file)
        op_files.append(op_file)
        counter = counter + 1
    return op_files
    
def slice_spect(verbose=0, mode=None):
    if mode=="Train":
        if os.path.exists('/content/drive/MyDrive/IR Project/Train_Sliced_Images'):
            return
        labels = []
        image_folder = "/content/drive/MyDrive/IR Project/Train_Spectogram_Images"
        filenames = [os.path.join(image_folder, f) for f in os.listdir(image_folder)
                       if f.endswith(".jpg")]
        counter = 0
        if(verbose > 0):
            print("Slicing Spectograms ...")
        if not os.path.exists('/content/drive/MyDrive/IR Project/Train_Sliced_Images'):
            os.makedirs('/content/drive/MyDrive/IR Project/Train_Sliced_Images')
        for f in tqdm(filenames):
            genre_variable = re.search('/content/drive/MyDrive/IR Project/Train_Spectogram_Images/.*_(.+?).jpg', f).group(1)
            img = Image.open(f)
            subsample_size = 128
            width, height = img.size
            number_of_samples = width // subsample_size
            for i in range(number_of_samples):
                start = i*subsample_size
                img_temporary = img.crop((start, 0., start + subsample_size, subsample_size))
                img_temporary.save("/content/drive/MyDrive/IR Project/Train_Sliced_Images/"+str(counter)+"_"+genre_variable+".jpg")
                counter = counter + 1
        return

    elif mode=="Test":
        if os.path.exists('/content/drive/MyDrive/IR Project/Test_Sliced_Images'):
            return
        labels = []
        image_folder = "/content/drive/MyDrive/IR Project/Test_Spectogram_Images"
        filenames = [os.path.join(image_folder, f) for f in os.listdir(image_folder)
                       if f.endswith(".jpg")]
        counter = 0
        if(verbose > 0):
            print("Slicing Spectograms ...")
        if not os.path.exists('/content/drive/MyDrive/IR Project/Test_Sliced_Images'):
            os.makedirs('/content/drive/MyDrive/IR Project/Test_Sliced_Images')
        for f in filenames:
            song_variable = re.search('/content/drive/MyDrive/IR Project/Test_Spectogram_Images/(.+?).jpg', f).group(1)
            img = Image.open(f)
            subsample_size = 128
            width, height = img.size
            number_of_samples = width // subsample_size
            for i in range(number_of_samples):
                start = i*subsample_size
                img_temporary = img.crop((start, 0., start + subsample_size, subsample_size))
                img_temporary.save("/content/drive/MyDrive/IR Project/Test_Sliced_Images/"+str(counter)+"_"+song_variable+".jpg")
                counter = counter + 1
        return

In [6]:
def get_data(inp_file):
    print("Creating Spectogram...")
    op_file = create_spectrogram_file(inp_file)
    print("Slicing Spectogram...")
    filenames = slice_spect_file(op_file)
    print("Converting to numpy array")
    images_all = [None]*(len(filenames))
    index = 0
    for f in filenames:
        temp = cv2.imread(f, cv2.IMREAD_UNCHANGED)
        images_all[index] = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
        index += 1
    images = np.array(images_all)
    return images

def get_data_online_url(url):
    print("Fetching File from URL...")
    z = io.BytesIO(urlopen(url).read())
    file_path = '/content/drive/MyDrive/IR Project/Data/test/'+url.split("/")[-1]
    pathlib.Path(file_path).write_bytes(z.getbuffer())
    return get_data(file_path)

def load_dataset(verbose=0, mode=None, datasetSize=1.0):
    if os.path.exists('/content/drive/MyDrive/IR Project/Training_Data'):
            train_x = np.load("/content/drive/MyDrive/IR Project/Training_Data/train_x.npy")
            train_y = np.load("/content/drive/MyDrive/IR Project/Training_Data/train_y.npy")
            test_x = np.load("/content/drive/MyDrive/IR Project/Training_Data/test_x.npy")
            test_y = np.load("/content/drive/MyDrive/IR Project/Training_Data/test_y.npy")
            n_classes, genre_new = joblib.load("/content/drive/MyDrive/IR Project/Training_Data/meta.pkl")
            return train_x, train_y, test_x, test_y, n_classes, genre_new
    create_spectrogram(verbose, mode)
    slice_spect(verbose, mode)

    # datasetSize is a float value which returns a fraction of the dataset.
    # If set as 1.0 it returns the entire dataset.
    # If set as 0.5 it returns half the dataset.

    if mode=="Train":
        genre = {
        "Hip-Hop": 0,
        "International": 1,
        "Electronic": 2,
        "Folk" : 3,
        "Experimental": 4,
        "Rock": 5,
        "Pop": 6,
        "Instrumental": 7
        }
        if(verbose > 0):
            print("Compiling Training and Testing Sets ...")
        filenames = [os.path.join("/content/drive/MyDrive/IR Project/Train_Sliced_Images", f) for f in os.listdir("/content/drive/MyDrive/IR Project/Train_Sliced_Images")
                       if f.endswith(".jpg")]
        images_all = [None]*(len(filenames))
        labels_all = [None]*(len(filenames))
        for f in tqdm(filenames):
            index = int(re.search('/content/drive/MyDrive/IR Project/Train_Sliced_Images/(.+?)_.*.jpg', f).group(1))
            genre_variable = re.search('/content/drive/MyDrive/IR Project/Train_Sliced_Images/.*_(.+?).jpg', f).group(1)
            temp = cv2.imread(f, cv2.IMREAD_UNCHANGED)
            images_all[index] = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
            labels_all[index] = genre[genre_variable]

        if(datasetSize == 1.0):
            images = images_all
            labels = labels_all

        else:
            count_max = int(len(images_all)*datasetSize / 8.0)
            count_array = [0, 0, 0, 0, 0 ,0, 0, 0]
            images = []
            labels = []
            for i in range(0, len(images_all)):
                if(count_array[labels_all[i]] < count_max):
                    images.append(images_all[i])
                    labels.append(labels_all[i])
                    count_array[labels_all[i]] += 1
            images = np.array(images)
            labels = np.array(labels)

        images = np.array(images)
        labels = np.array(labels)
        labels = labels.reshape(labels.shape[0],1)
        train_x, test_x, train_y, test_y = train_test_split(images, labels, test_size=0.05, shuffle=True)

        # Convert the labels into one-hot vectors.
        train_y = np_utils.to_categorical(train_y)
        test_y = np_utils.to_categorical(test_y, num_classes=8)
        n_classes = len(genre)
        genre_new = {value: key for key, value in genre.items()}

        if not os.path.exists('/content/drive/MyDrive/IR Project/Training_Data'):
            os.makedirs('/content/drive/MyDrive/IR Project/Training_Data')
        np.save("/content/drive/MyDrive/IR Project/Training_Data/train_x.npy", train_x)
        np.save("/content/drive/MyDrive/IR Project/Training_Data/train_y.npy", train_y)
        np.save("/content/drive/MyDrive/IR Project/Training_Data/test_x.npy", test_x)
        np.save("/content/drive/MyDrive/IR Project/Training_Data/test_y.npy", test_y)
        joblib.dump((n_classes, genre_new), "/content/drive/MyDrive/IR Project/Training_Data/meta.pkl")
        return train_x, train_y, test_x, test_y, n_classes, genre_new

    if mode=="Test":
        if(verbose > 0):
            print("Compiling Training and Testing Sets ...")
        filenames = [os.path.join("/content/drive/MyDrive/IR Project/Test_Sliced_Images", f) for f in os.listdir("/content/drive/MyDrive/IR Project/Test_Sliced_Images")
                       if f.endswith(".jpg")]
        images = []
        labels = []
        for f in filenames:
            song_variable = re.search('/content/drive/MyDrive/IR Project/Test_Sliced_Images/.*_(.+?).jpg', f).group(1)
            tempImg = cv2.imread(f, cv2.IMREAD_UNCHANGED)
            images.append(cv2.cvtColor(tempImg, cv2.COLOR_BGR2GRAY))
            labels.append(song_variable)

        images = np.array(images)

        return images, labels

In [7]:
train_x, train_y, test_x, test_y, n_classes, genre = load_dataset(verbose=1, mode="Train", datasetSize=0.75)
# datasetSize = 0.75, this returns 3/4th of the dataset.

# Expand the dimensions of the image to have a channel dimension. (nx128x128) ==> (nx128x128x1)
train_x = train_x.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2], 1)
test_x = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2], 1)

# Normalize the matrices.
train_x = train_x / 255.
test_x = test_x / 255.

yes


In [8]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=[7,7], kernel_initializer = initializers.he_normal(seed=1), activation="relu", input_shape=(128,128,1)))
# Dim = (122x122x64)
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=[2,2], strides=2))
# Dim = (61x61x64)
model.add(Conv2D(filters=128, kernel_size=[7,7], strides=2, kernel_initializer = initializers.he_normal(seed=1), activation="relu"))
# Dim = (28x28x128)
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=[2,2], strides=2))
# Dim = (14x14x128)
model.add(Conv2D(filters=256, kernel_size=[3,3], kernel_initializer = initializers.he_normal(seed=1), activation="relu"))
# Dim = (12x12x256)
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=[2,2], strides=2))
# Dim = (6x6x256)
model.add(Conv2D(filters=512, kernel_size=[3,3], kernel_initializer = initializers.he_normal(seed=1), activation="relu"))
# Dim = (4x4x512)
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=[2,2], strides=2))
# Dim = (2x2x512)
model.add(BatchNormalization())
model.add(Flatten())
# Dim = (2048)
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(1024, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (1024)
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (256)
model.add(Dropout(0.25))
model.add(Dense(64, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (64)
model.add(Dense(32, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (32)
model.add(Dense(n_classes, activation="softmax", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (8)
print(model.summary())
# plot_model(model, to_file="Saved_Model/Model_Architecture.jpg")
model.compile(loss="categorical_crossentropy", optimizer=tf.optimizers.Adam(lr=0.0001), metrics=['accuracy'])
pd.DataFrame(model.fit(train_x, train_y, epochs=30, verbose=1, validation_split=0.1).history).to_csv("/content/drive/MyDrive/IR Project/Saved_Model/training_history.csv")
score = model.evaluate(test_x, test_y, verbose=1)
print(score)
model.save("/content/drive/MyDrive/IR Project/Saved_Model/Model.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 122, 122, 64)      3200      
                                                                 
 batch_normalization (BatchN  (None, 122, 122, 64)     256       
 ormalization)                                                   
                                                                 
 average_pooling2d (AverageP  (None, 61, 61, 64)       0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 128)       401536    
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 128)      512       
 hNormalization)                                                 
                                                        

In [9]:
def get_genre(url, online=True):
    model = keras.models.load_model("/content/drive/MyDrive/IR Project/Saved_Model/Model.h5")
    genre = {
        0: "Hip-Hop",
        1: "International",
        2: "Electronic",
        3: "Folk",
        4: "Experimental",
        5: "Rock",
        6: "Pop",
        7: "Instrumental"
        }
    if online:
        inputs = get_data_online_url(url)
    else:
        inputs = get_data(url)
    outputs = model(inputs)
    output = np.argmax(outputs, axis=1)
    return [genre[label] for label in output]

def get_embeddings(url, online=True):
    raw_model = keras.models.load_model("/content/drive/MyDrive/IR Project/Saved_Model/Model.h5")
    model = Model(inputs=raw_model.input, outputs=raw_model.layers[-2].output)
    if online:
        inputs = get_data_online_url(url)
    else:
        inputs = get_data(url)
    outputs = model(inputs)
    output = np.mean(outputs, axis=0)
    return output

In [10]:
url = "/content/drive/MyDrive/IR Project/Data/test/the-cradle-of-your-soul-15700.mp3"

In [11]:
get_embeddings(url, online=False)

Creating Spectogram...
Slicing Spectogram...
Converting to numpy array


array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       2.2191207e+04, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.3199707e+04, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 2.9160959e+04, 0.0000000e+00,
       5.8543970e+03, 0.0000000e+00, 0.0000000e+00, 6.2868921e+03,
       0.0000000e+00, 0.0000000e+00, 1.3185873e+04, 9.4970398e+00,
       1.2568900e+04, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 2.3969320e+04, 0.0000000e+00, 0.0000000e+00],
      dtype=float32)

In [12]:
get_genre(url, online=False)

Creating Spectogram...
Slicing Spectogram...
Converting to numpy array


['Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental',
 'Experimental']